In [265]:
using PyPlot
using Optim
using ImageFiltering
using Interpolations

In [266]:
# Convert an Float32 rbg image to Float64 grayscale image
function convert_to_grayscale(I::Array{Float32,3})
    I=convert(Array{Float64,3}, I)
    I_gray = 0.2989*I[:,:,1] + 0.5870*I[:,:,2] + 0.1140*I[:,:,3]
    return I_gray::Array{Float64,2}
end

convert_to_grayscale (generic function with 1 method)

In [267]:
# Load Tsukuba disparity dataset and convert it to grayscale
function load_data()
    i0_path = string(@__DIR__,"/i0.png")
    i0 = imread(i0_path)
    i0 = convert_to_grayscale(i0)
    i1_path = string(@__DIR__,"/i1.png")
    i1 = imread(i1_path)
    i1 = convert_to_grayscale(i1)
    gt_path = string(@__DIR__,"/gt.png")
    gt64 = convert(Array{Float64,2}, imread(gt_path)*255)

    @assert maximum(gt64) <= 16
    return i0::Array{Float64,2}, i1::Array{Float64,2}, gt64::Array{Float64,2}
end

load_data (generic function with 1 method)

In [268]:
function log_studentt(x::Float64, alpha::Float64, sigma::Float64)
    
    #TODO: Change the code to accept matrix input
    
    function log_student(x, alpha, sigma)
        return log((1.0 + (1.0/(2.0*sigma^2))*x^2)^(-1.0*alpha))
    end
    
    function log_grad_student(x, alpha, sigma)
        return -(2.0*alpha*x)/((2.0*sigma^2)+x^2)
    end
    
    value = log_student(x, alpha, sigma)
    grad = log_grad_student(x, alpha, sigma)

    return value::Float64, grad::Float64
end

log_studentt (generic function with 1 method)

In [269]:
# Evaluate stereo log prior.
# Set: alpha=1.0, sigma=1.0
function stereo_log_prior(x::Array{Float64,2})
    alpha = 1.0
    sigma = 1.0
    
    value = 0
    grad = zeros(size(x))
    for i = 1:size(x)[1]-1#Height?
        for j = 1:size(x)[2]-1#Width?
            value += log_studentt(x[i+1, j] - x[i,j], alpha, sigma)[1]
            value += log_studentt(x[i, j+1] - x[i,j], alpha, sigma)[1]
        end
    end
    
    # Partial derivative to every pixel
    for k = 1:size(x)[1]
        for l = 1:size(x)[2]
            if k + 1 <= size(x)[1]
                grad[k,l] += log_studentt(x[k,l]-x[k+1,l], alpha, sigma)[2] 
            end
            if k - 1 >= 1
                grad[k,l] += log_studentt(x[k,l]-x[k-1,l], alpha, sigma)[2]
            end
            if l + 1 <= size(x)[2]
                grad[k,l] += log_studentt(x[k,l]-x[k,l+1], alpha, sigma)[2]
            end
            if l - 1 >= 1
                grad[k,l] += log_studentt(x[k,l]-x[k,l-1], alpha, sigma)[2]
            end
        end
    end
    
    return  value::Float64, grad::Array{Float64,2}
end


stereo_log_prior (generic function with 1 method)

In [270]:
# Evaluate stereo log likelihood.
# Set: Alpha = 1.0, Sigma = 0.004
function stereo_log_likelihood(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})
    alpha = 1.0
    sigma = 0.004
    value = 0
    grad = zeros(size(im1))
    
    # We need the horizontal image derivative from I1 to calculate the gradient of the LH
    kernel = centered([1.0 0.0 -1.0; 2.0 0.0 -2.0; 1.0 0.0 -1.0]);
    sobel_x = imfilter(im1, kernel);
    h_img_grad = imfilter(sobel_x, kernel')    
    
    # We need to interpolate im1 and the horizontal image derivative,
    # as we use continous optimization and might have continous disparities now
    im1 = interpolate(im1, BSpline(Linear()))
    h_img_grad = interpolate(h_img_grad,  BSpline(Linear()))


    for i = 1:size(x)[1]
        for j = 1:size(x)[2]
            # Clamp disparity values to (0, 14.0)
            # TODO: Use bounded optimization instead
            disp = clamp(x[i,j], 0.0, 14.0)
            # Substract value of Im0 from value of (left) shifted Im1
            d = im0[i,j]-im1[i-disp, j]
            value += log_studentt(d, alpha, sigma)[1]
            grad[i,j] = (-1)*log_studentt(d, alpha, sigma)[2]*h_img_grad[i-disp, j]          
        end
    end
    return value::Float64, grad::Array{Float64,2}
end

stereo_log_likelihood (generic function with 1 method)

In [271]:
# Evaluate stereo posterior
function stereo_log_posterior(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2}) 
    #log(posterior) = log(prior*LH) = log(prior) + log(LH)
    # (We can drop the marginalisation terms)
    log_posterior = stereo_log_likelihood(x, im0, im1)[1] + stereo_log_prior(x)[1] #+ stereo_log_prior(im0)[1]
    grad_lh = stereo_log_likelihood(x, im0, im1)[2]
    grad_x = stereo_log_prior(x)[2]
    # (Derivative of I0 to x should be zero, so we drop it ...)
    log_posterior_grad = grad_lh + grad_x
    
    # Instead of maximizing the log (to 0), we want to minimize the - log (to 0)
    return -log_posterior::Float64, -log_posterior_grad::Array{Float64,2}
end

stereo_log_posterior (generic function with 1 method)

In [272]:
# Run stereo algorithm using gradient ascent or sth similar
function stereo(x0::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})
   
    # Helper function with fixed im0 and im1
    function f(y)
        value = stereo_log_posterior(y, im0, im1)[1]
        return value
    end
    
    # Helper function for gradients
    function g!(storage, y)
        grad = stereo_log_posterior(y, im0, im1)[2]
        storage[:,:] = grad
    end
    
    #=
    
    ---------- Boxed Optimization ----------
    
    # Minimum 0 disparity
    lower = zeros(size(x0))

    # Maximum 14.0 disparity
    upper = fill(14.0, size(x0))

    od = OnceDifferentiable(f, g!, x0)
    
    res = optimize(od, lower, upper, x0, Fminbox(GradientDescent()), options)
    =#
    
    options = Optim.Options(iterations=10, show_trace=true, allow_f_increases=false)
    # Specify optim algorithm here
    res = optimize(f, g!, x0, ConjugateGradient(), options)
    
    x = Optim.minimizer(res)
    return x::Array{Float64,2}
end

stereo (generic function with 1 method)

In [273]:
# create constant disparity of all 8's of size DISPARITY_SIZE
function constant_disparity(disparity_size::Tuple{Int64,Int64})
    disparity_map = fill(8.0, disparity_size)
    return disparity_map::Array{Float64,2}
end

constant_disparity (generic function with 1 method)

In [274]:
# Create random disparity in [0,14] of size DISPARITY_SIZE
# We changed DISPARITY_SIZE to a tuple of integers
function random_disparity(disparity_size::Tuple{Int64,Int64})
    disparity_map = Array{Float64,2}(rand(collect(1:14),disparity_size))
    return disparity_map::Array{Float64,2}
end

random_disparity (generic function with 1 method)

In [275]:
function problem3()
    # use problem 2's load_data
    im0, im1, gt = load_data()
    
    # Display stereo: Initialized with constant 8's
    const_d = constant_disparity(size(gt))
    
    # Display stereo: Initialized with noise in [0,14]
    random_d = random_disparity(size(gt))
    
    # print("Prior GT disparity: ", stereo_log_prior(gt)[1])
    # println("\n Prior constant disparity: ", stereo_log_prior(const_d)[1])
    # println("\n Prior random disparity: ", stereo_log_prior(random_d)[1])
    # print("\nLH const disp: ", stereo_log_likelihood(const_d, im0, im1)[1])
    x = stereo(random_d, im0, im1)
    ## println("We got here")
    #figure()
    #title("Estimated disparity map")
    #imshow(x, "gray")
    #show()
    #figure()
    #title("Ground truth disparity map")
    #imshow(gt, "gray")
    #show()
    #imshow(shift_disparity(im0, gt), "gray")
    
    

    # print("\nLH random disp: ", stereo_log_likelihood(random_d, im0, im1)[1])
    
    # Display stereo: Initialized with gt
    # print("\nLH GT disp: ", stereo_log_likelihood(gt, im0, im1)[1])
    
    # Coarse to fine estimation..

end


problem3 (generic function with 1 method)

In [276]:
problem3()

┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = stereo_log_likelihood(::Array{Float64,2}, ::Array{Float64,2}, ::Array{Float64,2}) at In[270]:26
└ @ Main ./In[270]:26


BoundsError: BoundsError: attempt to access 288×384 interpolate(::Array{Float64,2}, BSpline(Linear())) with element type Float64 at index [-11.0, 1]